# Chapter 3: Fibonacci Example–Part 1
The best learning is by doing. We will now walk through the [fibonacci.py](https://github.com/qwang98/PyChiquito/blob/main/pychiquito/fibonacci.py) example with the following PLONKish table layout and three signals–"a", "b", and "c".
| a | b | c |
| - | - | - |
| 1 | 1 | 2 |
| 1 | 2 | 3 |
| 2 | 3 | 5 |
| 3 | 5 | 8 |
| ... | |   |
## Imports
These imports are for the typing hints included in this example. Python is a dynamically typed interpreted language and typings aren't enforced.

In [41]:
from __future__ import annotations
from typing import Tuple

The following imports are required, including:
- `Circuit` and `StepType`, the most important data types, from the domain specific language (dsl).
- Equal constraint `eq` from the constraint builder (cb).
- Field element `F` from utils.

In [42]:
from dsl import Circuit, StepType
from cb import eq
from util import F

## Circuit
On the highest level, we are building a `Circuit` object in PyChiquito, that's why we start with creating a `Fibonacci` class that inherits the `Circuit` parent class. Within the `Fibonacci` class, we define two functions:
- `setup`, which defines the circuit configuration using signals and step types (more on this later).
- `trace`, which defines the circuit layout and the trace of assigning witness values.

## Circuit Setup
We first define the circuit `setup`:

In [43]:
class Fibonacci(Circuit):
    def setup(self):
        self.a = self.forward("a")
        self.b = self.forward("b")
        
        self.fibo_step = self.step_type(FiboStep(self, "fibo_step"))

    def trace(self):
        pass

We `setup` circuit configuration using signals and step types. "Signals" are variables we use to express custom constraints and lookup arguments. PyChiquito is a step-based language, for which we configure different "step types". Each step type defines different relationships among the signals. Each circuit is an arbitrary combination of step type instances. Think of the step types as different burger ingredients, such as bun, lettuce, and patties. Building a Chiquito circuit is essentially combining these ingredients to make a burger. For example, 1 bun, 1 lettuce, 2 patties, and 1 bun. It's that simple!

Now back to signals. We can assign different values to the same signal at different step instances, we can query the signal at different positions, and that's why signals are also called `Queriable` in PyChiquito. There are signals that live on the circuit top-level, called "forward signals", which we can query at different step instances. There are signals that live in a specific step instance, called "internal signals".

In the example above, we add two signals, "a" and "b", to the Fibonacci circuit. Again, we append signals to the circuit by defining them as `self.a` and `self.b`. Because they are circuit top-level signals rather than specific to a step-type, they are forward signals created using `self.forward`.

The example above only involves one step type. Here, step type definition starts with `self.fibo_step`, because we append it directly to the circuit. `step_type` function only has one argument, which is the `FiboStep` object created using its class constructor.

Before we dive into `trace`, let's define the `FiboStep` class first.

## StepType
PyChiquito provides a `StepType` parent class that we can customarily inherit. Again, for each `StepType`, we define two functions:
- `setup`, which defines the step type configuration using signals
- `wg`, which defines witness assignment within the step type

## FiboStep Setup
In the Fibonacci cicuit, we consider each row a step instance. For example, in row 1, a=1 b=1 c=2. In row 2, a=1 b=2 c=3. Given the nature of Fibonacci, We want to enforce the constraint that b in the current step instance equals a in the next step instance and that c in the current step instance equals b in the next step instance. We created "a" and "b" as forward signals above, because they are referred to across multiple step instances.

We now define the only step type, `FiboStep`:

In [44]:
class FiboStep(StepType):
    def setup(self):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))
        self.transition(eq(self.circuit.b, self.circuit.a.next()))
        self.transition(eq(self.c, self.circuit.b.next()))

    def wg(self):
        pass

As each step instance contains three signals, only two of which we've defined so far, we define a third signal "c", whose relationship is only defined within the step type and therefore an internal signal created using `self.internal`.

Next, we define constraints among signals, both forward and internal. There are two types of constraints in PyChiquito:
- `constr` stands for constraints among signals that are specific to the step type, i.e. internal signals.
- `transition` stands for constraints involving circuit-level signals, i.e. forward signals and etc.

In the code snippet below, forward signals "a" and "b" are expressed as `self.circuit.a` and `self.circuit.b`, whereas internal signal "c" is expressed `self.c`, because "a" and "b" are at the circuit-level. `self.circuit.a.next()` queries the value of circuit-level signal "a" at the next step instance. `eq` is a constraint builder that enforces equality between the two arguments passed in. The following constraints are translated as:
- a + b == c
- b == value of a in next step instance
- c = value of b in next step instance

## FiboStep Witness Generation

In [45]:
class FiboStep(StepType):
    def setup(self: FiboStep):
        pass

    def wg(self: FiboStep, args: Tuple[int, int]):
        a_value, b_value = args
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

In the example above, `wg` (witness generation) defines witness value assignments at the step type level. Here, the `args` we pass in is a tuple of values for signals "a" and "b". We assign them to forward signals "a" and "b" and then their sum to internal signal "c".

Finally, note that in `self.assign`, `a_value` and `b_value` are both wrapped in `F`, which converts them from int to field elements. All witness assignments in PyChiquito are field elements.

Putting everything for `FiboStep` together, we have:

In [46]:
class FiboStep(StepType):
    def setup(self):
        self.c = self.internal("c")
        self.constr(eq(self.circuit.a + self.circuit.b, self.c))
        self.transition(eq(self.circuit.b, self.circuit.a.next()))
        self.transition(eq(self.c, self.circuit.b.next()))

    def wg(self, args):
        a_value, b_value = args
        self.assign(self.circuit.a, F(a_value))
        self.assign(self.circuit.b, F(b_value))
        self.assign(self.c, F(a_value + b_value))

## Circuit Trace
Now we finished defining the ingredient (step type). Let's make the burger (combining step instances)!

In [47]:
class Fibonacci(Circuit):
    def setup(self):
        self.a = self.forward("a")
        self.b = self.forward("b")

        self.fibo_step = self.step_type(FiboStep(self, "fibo_step"))
        
    def trace(self, args):
        self.add(self.fibo_step, (1, 1))
        a = 1
        b = 2
        for i in range(1, 10):
            self.add(self.fibo_step, (a, b))
            prev_a = a
            a = b
            b += prev_a

In `trace`, we define the circuit layout using step instances and a trace of witness value assignments. `trace` takes two arguments, the `Fibonacci` circuit itself and the witness value assignment arguments `args`.

We call `self.add` to instantiate `fibo_step` we defined and pass in the witness values for "a" and "b". Note that we only hardcoded witness values for the first step instance as `(1, 1)`, because all other witness values can be calculated given the nature of Fibonacci. 

Note that `self.add` creates step instance by calling `wg` associated with the step type. Therefore `args` in `wg`, i.e. tuple of `a_value, b_value`, needs to match the second input of `self.add`, e.g. `(a, b)` in `self.add(self.fibo_step, (a, b))`.

We didn't pass in witness values for "c", because they are only ever calculated in `wg`.

Note that we need to pass in witness value assignments in a single argument `args` and therefore we use a tuple in this case. `args` can really be any data type as long as it's one single argument.

After creating the first `fibo_step` instance, we loop over `fibo_step` creation for 9 more times, each time calculating and passing in a different tuple of assignments. Voila, here's our PyChiquito Fibonacci "burger" with 10 patties (`fibo_step`) and nothing else!

## Putting Everything Together
Everything we went through above defines how the circuit and its step type are configured and witness values assigned to them. To instantiate the circuit, we call the class constructor:

In [48]:
fibo = Fibonacci()

You can also print the circuit. In the print out, you will see the single step type and two forward signals "a" and "b" at the circuit top-level. Within each step type, you will see the internal signals and constraints. The big random looking numbers are UUIDs that we use to uniquely identify objects in the circuit, which you don't need to worry about.

In [49]:
print(fibo)

ASTCircuit(
	step_types={
		246502402039451354789087853155778693642: ASTStepType(
			id=246502402039451354789087853155778693642,
			name='fibo_step',
			signals=[
				InternalSignal(id=246502415508238982210296263566787480074, annotation='c')
			],
			constraints=[
				Constraint(
					annotation='((a + b) == c)',
					expr=(a + b - (-c))
				)
			],
			transition_constraints=[
				TransitionConstraint((b == next(a))),
				TransitionConstraint((c == next(b)))
			],
			annotations={
				246502415508238982210296263566787480074: c
			}
		)
	},
	forward_signals=[
		ForwardSignal(id=246502365594496598225421467246924532234, phase=0, annotation='a'),
		ForwardSignal(id=246502388570663727361143183602130487818, phase=0, annotation='b')
	],
	shared_signals=[],
	fixed_signals=[],
	exposed=[],
	annotations={
		246502365594496598225421467246924532234: a,
		246502388570663727361143183602130487818: b,
		246502402039451354789087853155778693642: fibo_step
	},
	fixed_gen=None,
	first_step=None,
	last_ste

After initiating the Fibonacci circuit, we can generate witness assignments for it. `gen_witness` takes one argument of external input with `Any` type. However, because the only external input, `(1, 1)`, was hardcoded in `trace`, we don't need to provide an additional one and can put `None` for this argument. In practice, one circuit can have many different sets of witness assignments, each generated by a different external input argument. This is why we expose the `gen_witness` function to you.

In [50]:
fibo_witness = fibo.gen_witness(None)

Again, you can print the witness assignments:

In [51]:
print(fibo_witness)

TraceWitness(
	step_instances={
		StepInstance(
			step_type_uuid=246502402039451354789087853155778693642,
			assignments={
				a = 1,
				b = 1,
				c = 2
			},
		),
		StepInstance(
			step_type_uuid=246502402039451354789087853155778693642,
			assignments={
				a = 1,
				b = 2,
				c = 3
			},
		),
		StepInstance(
			step_type_uuid=246502402039451354789087853155778693642,
			assignments={
				a = 2,
				b = 3,
				c = 5
			},
		),
		StepInstance(
			step_type_uuid=246502402039451354789087853155778693642,
			assignments={
				a = 3,
				b = 5,
				c = 8
			},
		),
		StepInstance(
			step_type_uuid=246502402039451354789087853155778693642,
			assignments={
				a = 5,
				b = 8,
				c = 13
			},
		),
		StepInstance(
			step_type_uuid=246502402039451354789087853155778693642,
			assignments={
				a = 8,
				b = 13,
				c = 21
			},
		),
		StepInstance(
			step_type_uuid=246502402039451354789087853155778693642,
			assignments={
				a = 13,
				b = 21,
				c = 34
			},
		),
		StepInstance(
	

We can convert and register the PyChiquito circuit as a Halo2 circuit in Rust Chiquito. `ast_to_halo2` function achieves this purpose and prints out the Halo2 circuit UUID generated for our PyChiquito circuit. You don't need to do anything with the UUID.

In [52]:
fibo.ast_to_halo2()

253529837850136958128877792815148042762
253529837850136958128877792815148042762


Finally, we can generate and verify the proof with the witness. The print out includes Halo2 and Rust Chiquito debug messages. `Ok(())` means that proof was correctly generated and verified for the witness assignments and circuit. `Err()` prints out Halo2 and Rust Chiquito error messages, usually because some constraints in the circuit were not satisfied. Here, you should see the `Ok(())` print out.

In [53]:
fibo.verify_proof(fibo_witness)

Ok(
    (),
)


Congratulations! Now you finished writing your first Fibonacci circuit and learned about the most essential concepts behind the step-based design of PyChiquito, which is really as easy as bun-lettuce-patty-patty-bun, basically combining step instances to a PyChiquito burger! With abstraction, composability, modularity, and smooth user experience as the key tenets, writing Halo2 circuits has never been easier with PyChiquito!

Next up, in Chapter 3: Fibonacci Example-Part 2, you will learn about testing your circuit with multiple different witnesses.